In [1]:
import os
import yaml
import psycopg2
import pandas as pd
import urllib.request
from sqlalchemy import text
from sqlalchemy import create_engine
import time
import random
import numpy as np
import scipy.sparse as sparse

In [2]:
# setting WD
os.chdir('/Users/Daniel.Kwon/projects')

In [5]:
# database connection info
with open("./redshift_credentials.yml", 'r') as stream:
    try:
        rs_cred = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
engine_url = "postgresql://{user_name}:{pw}@{host}:{port}/{db_name}".format(user_name = rs_cred['default']['user'],
                                                                            pw        = urllib.parse.quote_plus(rs_cred['default']['password']),
                                                                            host      = rs_cred['default']['host'],
                                                                            port      = rs_cred['default']['port'],
                                                                            db_name   = rs_cred['default']['dbname']
                                                                           )
engine = create_engine(engine_url)

In [6]:
# sessions
query_1 = open('./recommender-systems/notebooks/user-item-viewcount.sql', 'r')
query_text_1 = text(query_1.read())
sessions = pd.read_sql_query(query_text_1, engine)
# products
query_2 = open('./recommender-systems/notebooks/items.sql', 'r')
query_text_2 = text(query_2.read())
products = pd.read_sql_query(query_text_2, engine)

In [7]:
sessions.head()

,ip_address,user_id,product_id,total_views
0,34.209.195.77,1961732978819,4578206416951,2749
1,34.209.195.77,34902656255133,4578205958199,188
2,104.162.245.74,4726733716754646,4562785796151,154
3,74.208.19.71,57562200693494,1719134781495,127
4,72.34.90.74,585684219541414,4578206416951,92


In [31]:
users = list(np.sort(sessions.user_id.unique()))
items = list(np.sort(sessions.user_id.unique()))
views = list(sessions.total_views)

In [33]:
print(len(sessions.product_id.unique()))
print(len(items))

900
12598026


In [26]:
# Get the rows and columns for our new matrix
products_rows = sessions.product_id.astype(int)
users_cols = sessions.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((views, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

ValueError: row index exceeds matrix dimensions

In [ ]:
sparse_product_user

In [11]:
# create zero-based index position <-> user/item ID mappings
index_to_user = pd.Series(users)

# create reverse mappings from user/item ID to index positions
user_to_index = pd.Series(data=index_to_user.index + 1, index=index_to_user.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = sessions.product_id.astype(int)
users_cols = sessions.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((views, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

sparse_user_product = sparse.csr_matrix((views, (users_cols, products_rows)), shape=(len(users) + 1, len(items) + 1))
sparse_user_product.data = np.nan_to_num(sparse_user_product.data, copy=False)

ValueError: row index exceeds matrix dimensions

In [34]:
sparse_product_user

0              52.13.144.5
1              52.13.144.5
2             173.59.44.20
3           208.87.236.201
4            173.69.16.202
                 ...      
47785547     68.187.47.250
47785548       68.187.5.18
47785549       68.187.5.18
47785550     68.187.50.226
47785551     68.187.56.173
Name: user_ip, Length: 47785552, dtype: object